In [0]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


### In the previous repository  we saved finetuned language model. Here we will csee how to use pretrained and finetuned language model for sequence classification and will compare them. 

In [0]:
!pip install pytorch-transformers
!pip install transformers

     |████████████████████████████████| 184kB 4.8MB/s 
     |████████████████████████████████| 1.0MB 9.8MB/s 
     |████████████████████████████████| 870kB 16.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=5f1aad702d1c846c9f661c48ad928b046ddb9d352fde7e8012807326611b3a4c
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 450kB 5.1MB/s 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
directory_path='/content/drive/My Drive/xlnet_experiments'

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


from pytorch_transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from pytorch_transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

Using TensorFlow backend.


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [0]:
df = pd.read_csv("/content/drive/My Drive/xlnet_experiments/ExperimentData/labeledTrainData.tsv", delimiter='\t')


In [0]:
df.shape

(25000, 3)

In [0]:
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [0]:
sentences = df.review.values


In [0]:
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.sentiment.values

In [0]:
sentences[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [0]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

100%|██████████| 798011/798011 [00:00<00:00, 2473629.87B/s]


Tokenize the first sentence:
['▁with', '▁all', '▁this', '▁stuff', '▁going', '▁down', '▁at', '▁the', '▁moment', '▁with', '▁', 'm', 'j', '▁', 'i', "'", 've', '▁started', '▁listening', '▁to', '▁his', '▁music', ',', '▁watching', '▁the', '▁odd', '▁documentary', '▁here', '▁and', '▁there', ',', '▁watched', '▁the', '▁', 'wi', 'z', '▁and', '▁watched', '▁moon', 'walker', '▁again', '.', '▁maybe', '▁', 'i', '▁just', '▁want', '▁to', '▁get', '▁a', '▁certain', '▁insight', '▁into', '▁this', '▁guy', '▁who', '▁', 'i', '▁thought', '▁was', '▁really', '▁cool', '▁in', '▁the', '▁eight', 'ies', '▁just', '▁to', '▁maybe', '▁make', '▁up', '▁my', '▁mind', '▁whether', '▁he', '▁is', '▁guilty', '▁or', '▁innocent', '.', '▁moon', 'walker', '▁is', '▁part', '▁biography', ',', '▁part', '▁feature', '▁film', '▁which', '▁', 'i', '▁remember', '▁going', '▁to', '▁see', '▁at', '▁the', '▁cinema', '▁when', '▁it', '▁was', '▁originally', '▁released', '.', '▁some', '▁of', '▁it', '▁has', '▁subtle', '▁messages', '▁about', '▁', 'm', 'j

In [0]:
MAX_LEN = 128

In [0]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [0]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


In [0]:

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)


In [0]:


#Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=56, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=56, test_size=0.2)

In [0]:

# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [0]:
# Load XLNEtForSequenceClassification, the pretrained XLNet model with a single linear classification layer on top. 

model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()

100%|██████████| 467042463/467042463 [00:14<00:00, 32112527.11B/s]


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [0]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

In [0]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # # Validation

  # # Put model in evaluation mode to evaluate loss on the validation set
  # model.eval()

  # # Tracking variables 
  # eval_loss, eval_accuracy = 0, 0
  # nb_eval_steps, nb_eval_examples = 0, 0

  # # Evaluate data for one epoch
  # for batch in validation_dataloader:
  #   # Add batch to GPU
  #   batch = tuple(t.to(device) for t in batch)
  #   # Unpack the inputs from our dataloader
  #   b_input_ids, b_input_mask, b_labels = batch
  #   # Telling the model not to compute or store gradients, saving memory and speeding up validation
  #   with torch.no_grad():
  #     # Forward pass, calculate logit predictions
  #     output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
  #     logits = output[0]
    
  #   # Move logits and labels to CPU
  #   logits = logits.detach().cpu().numpy()
  #   label_ids = b_labels.to('cpu').numpy()

  #   tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
  #   eval_accuracy += tmp_eval_accuracy
  #   nb_eval_steps += 1

  # print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))


Epoch:  25%|██▌       | 1/4 [11:38<34:55, 698.40s/it]

Train loss: 0.3256512345314026


Epoch:  50%|█████     | 2/4 [23:26<23:22, 701.45s/it]

Train loss: 0.2084632540166378


Epoch:  75%|███████▌  | 3/4 [35:15<11:43, 703.61s/it]

Train loss: 0.12945395572781562


Epoch: 100%|██████████| 4/4 [47:04<00:00, 705.13s/it]

Train loss: 0.0802508061721921


In [0]:
torch.save(model.state_dict(), directory_path+'/model_without_language_model.ckpt')

In [0]:
directory_path

'/content/drive/My Drive/xlnet_experiments'

In [0]:
# torch.save(model.state_dict(), directory_path+'/model_without_language_model.ckpt')
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(validation_dataloader):
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      # print (outputs)
      prediction = torch.argmax(outputs[0],dim=1)
      total += b_labels.size(0)
      correct+=(prediction==b_labels).sum().item()

In [0]:
print('Test Accuracy of the model on vla data is: {} %'.format(100 * correct / total))


Test Accuracy of the model on val data is: 88.06 %


#### Using pretrained language model our accuracy is 88.06. 

Now, we will use our finetuned language model

Using **finetuned language model**

In [0]:
from transformers import XLNetTokenizer,XLNetForSequenceClassification

In [0]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['▁with', '▁all', '▁this', '▁stuff', '▁going', '▁down', '▁at', '▁the', '▁moment', '▁with', '▁', 'm', 'j', '▁', 'i', "'", 've', '▁started', '▁listening', '▁to', '▁his', '▁music', ',', '▁watching', '▁the', '▁odd', '▁documentary', '▁here', '▁and', '▁there', ',', '▁watched', '▁the', '▁', 'wi', 'z', '▁and', '▁watched', '▁moon', 'walker', '▁again', '.', '▁maybe', '▁', 'i', '▁just', '▁want', '▁to', '▁get', '▁a', '▁certain', '▁insight', '▁into', '▁this', '▁guy', '▁who', '▁', 'i', '▁thought', '▁was', '▁really', '▁cool', '▁in', '▁the', '▁eight', 'ies', '▁just', '▁to', '▁maybe', '▁make', '▁up', '▁my', '▁mind', '▁whether', '▁he', '▁is', '▁guilty', '▁or', '▁innocent', '.', '▁moon', 'walker', '▁is', '▁part', '▁biography', ',', '▁part', '▁feature', '▁film', '▁which', '▁', 'i', '▁remember', '▁going', '▁to', '▁see', '▁at', '▁the', '▁cinema', '▁when', '▁it', '▁was', '▁originally', '▁released', '.', '▁some', '▁of', '▁it', '▁has', '▁subtle', '▁messages', '▁about', '▁', 'm', 'j

In [0]:
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [0]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)


In [0]:
#Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=56, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=56, test_size=0.2)
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


 I saved previously saved pytorch based retrained xlnet model inside a new directory.  namely **xlnet_exp_pytorch** isnide direcotry path. That was just done due to some Googgle drive space issue. 

In [0]:
# Load XLNEtForSequenceClassification, the pretrained XLNet model with a single linear classification layer on top. 

model = XLNetForSequenceClassification.from_pretrained(directory_path+"/xlnet_exp_pytorch", num_labels=2)
model.cuda()

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [0]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

In [0]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    

Epoch:  25%|██▌       | 1/4 [11:42<35:07, 702.40s/it]

Train loss: 0.408814075088501


Epoch:  50%|█████     | 2/4 [23:25<23:25, 702.61s/it]

Train loss: 0.2643543077945709


Epoch:  75%|███████▌  | 3/4 [35:08<11:42, 702.85s/it]

Train loss: 0.18644530738294124


Epoch: 100%|██████████| 4/4 [46:53<00:00, 703.30s/it]

Train loss: 0.12285377791821957


In [0]:
torch.save(model.state_dict(), directory_path+'/model_with_language_model.ckpt')
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(validation_dataloader):
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      # print (outputs)
      prediction = torch.argmax(outputs[0],dim=1)
      total += b_labels.size(0)
      correct+=(prediction==b_labels).sum().item()

In [0]:
print('Test Accuracy of the finetuned model on val data is: {} %'.format(100 * correct / total))


Test Accuracy of the finetuned model on val data is: 87.32 %


We saw a dip in the score using finetuned language model on our task. One reason is that XLNET has already been trained on English  and our dataset is itself in English, that deteriorated the results. Had our data been in say Hindi, French etc then this (using finetuned model) could have yielded better results. 

### Things we saw in these notebooks

1. Retraining XLnet language model from checkpoint or without a checkpoint

2. Converting Tensorflow based language model using transformers-cli from Pytorch transformers

3. Using pytorch-transformers to train sequence classifier